# Lab 5 (part 2): Auto-Differentiation

In [ ]:
%matplotlib inline

# Toy problem
Suppose we want to fit a third order polynomial to predict $y=\sin(x)$ in the interval $-\pi \le x \le \pi$.

Let's start out by generating some training data $\{y_i,x_i\}_{i=1}^{N}$, where $i$ iterates over the $N$ training samples.

In [ ]:
import torch
import math
from matplotlib import pyplot as plt
import numpy as np

dtype = torch.float
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
N = 20
x = torch.linspace(-math.pi, math.pi, N, dtype=dtype)
y = torch.sin(x)

plt.plot(x.numpy(),y.numpy(),'.')

# Manual derivatives
Now, let's fit the training data using a third order polynomial by minimizing the L2 norm.

Writing the i'th prediction as $y_{pred,i}=a+bx_i+cx_i^2+dx_i^3$, we would like our model to output $y_{pred,i} \approx y_i$.

Manually calculating the partial derivatives of the loss function w.r.t. $a$, $b$, $c$, and $d$ would not a big deal for this toy example. We just need to differentitate the loss function

$J = \sum{(y_{pred,i}-y_i)^2} = \sum{(a+bx_i+cx_i^2+dx_i^3 - y_i)^2}$

where we are summing over the training samples.

Applying the chain rule a few times, we can easily show that

$\frac{\partial J}{\partial a} = 2\sum{y_{pred,i}-y_i}$

$\frac{\partial J}{\partial b} = 2\sum{x_i(y_{pred,i}-y_i)}$

$\frac{\partial J}{\partial c} = 2\sum{x_i^2(y_{pred,i}-y_i)}$

$\frac{\partial J}{\partial d} = 2\sum{x_i^3(y_{pred,i}-y_i)}$

We now have what we need to fit the polynomial using gradient descent.

**Your task:** Fill in the gaps below (marked with `???`) and run the code.

In [ ]:
# Create random Tensors for weights.
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-4
for t in range(2000):
    # Forward pass: compute predicted y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # Compute gradients of loss w.r.t. a, b, c, and d
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = ???

    # Update weights using gradient descent
    a = a - learning_rate * grad_a
    b = b - learning_rate * grad_b
    c = c - learning_rate * grad_c
    d = d - ???

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')
plt.plot(x.numpy(),y.numpy(),'.',x.numpy(),y_pred.detach().numpy())

# Automatic differentiation
Manual differentiation can quickly get very hairy for large complex networks. Thankfully, we can use automatic differentiation to automate the computation of backward passes in our model (or neural network). The autograd package in PyTorch provides exactly this functionality. When using autograd, the forward pass of your model will define a computational graph; nodes in the graph will be Tensors, and edges will be functions that produce output Tensors from input Tensors. Backpropagating through this graph then allows you to easily compute gradients.

This sounds complicated, it’s pretty simple to use in practice. Each Tensor represents a node in a computational graph. If x is a Tensor that has x.requires_grad=True then x.grad is another Tensor holding the gradient of x with respect to some scalar value.

Here we use PyTorch Tensors and autograd to implement our fitting sine wave with third order polynomial example; now we don't need to manually implement the backward pass through the model.

**Your task:** Uncomment the correct `backward()` call and fill in the gaps marked with `???`.

In [ ]:
# Create random Tensors for weights.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.randn((), dtype=dtype, requires_grad=True)
b = torch.randn((), dtype=dtype, requires_grad=True)
c = torch.randn((), dtype=dtype, requires_grad=True)
d = torch.randn((), dtype=dtype, requires_grad=True)

learning_rate = 1e-4
for t in range(2000):
    # Forward pass: compute predicted y using operations on Tensors.
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of the loss with respect to all Tensors with requires_grad=True.
    # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
    # the gradient of the loss with respect to a, b, c, d respectively.

    # TASK: Uncomment the correct backward() call
    #y_pred.backward()
    #loss.backward()
    #x.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= ???

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = ???

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')
plt.plot(x.numpy(),y.numpy(),'.',x.numpy(),y_pred.detach().numpy())

# Defining New autograd Functions
Instead of writing the polynomial as $y=a+bx+cx^2+dx^3$, we might want to write it as $y=a+b P_3(c+dx)$ where $P_3(x)=\frac{1}{2}\left(5x^3-3x\right)$ is
the [*Legendre polynomial*](https://en.wikipedia.org/wiki/Legendre_polynomials) of degree three.

This implementation computes the forward pass using operations on PyTorch
Tensors, and uses PyTorch autograd to compute gradients.

All we need to enable autograd is to implement a backward pass that calculates the *local gradient*, $P_3'(x)=\frac{3}{2}\left(5x^2-1\right)$.

**Your task:** Fill in the gap marked with `???`.

In [ ]:
import torch
import math

class LegendrePolynomial3(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        return 0.5 * (5 * input ** 3 - 3 * input)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input ** 2 - 1)

# Create random Tensors for weights. For this example, we need
# 4 weights: y = a + b * P3(c + d * x), these weights need to be initialized
# not too far from the correct result to ensure convergence.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
b = torch.full((), -1.0, device=device, dtype=dtype, requires_grad=True)
c = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
d = torch.full((), 0.3, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-3
for t in range(2000):
    # To apply our Function, we use Function.apply method. We alias this as 'P3'.
    P3 = LegendrePolynomial3.apply

    # Forward pass: compute predicted y using operations; we compute
    # P3 using our custom autograd operation.
    y_pred = a + b * P3(???)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass.
    loss.backward()

    # Update weights using gradient descent
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} * P3({c.item()} + {d.item()} x)')
plt.plot(x.numpy(),y.numpy(),'.',x.numpy(),y_pred.detach().numpy())

# Chain rule in a neuron
**Question:** What's the point of defining a new autograd function, like we just did?

**Background:**

We can imagine a neural network as a massive computational graph. PyTorch automatically sets up that graph for us.

Let us say we have a “neuron” f in that computational graph with inputs $x$ and $y$ which outputs $z$.

We can easily compute the local gradients — differentiating $z$ with respect to $x$ and $y$ as $\frac{\partial z}{\partial x}$ and $\frac{\partial z}{\partial y}$.

From the forward pass, we obtain the loss ($L$).

When we start to work the loss backwards, we get the gradient of the loss from the layer above $\frac{\partial L}{\partial z}$

In order for the loss to be propagated backwards, we need to find $\frac{\partial L}{\partial x}$ and $\frac{\partial L}{\partial y}$.

These can be calculated using the chain rule, i.e., $\frac{\partial L}{\partial x} = \frac{\partial L}{\partial z}\frac{\partial z}{\partial x}$ and $\frac{\partial L}{\partial y} = \frac{\partial L}{\partial z}\frac{\partial z}{\partial y}$

![picture](https://miro.medium.com/v2/resize:fit:720/format:webp/1*NpYmr8RwZ3x32GTBUkbB4A.png)

**Answer:**

All we need to build computational graphs that enable automatic differentiation is to define a **forward** function and a **backward** function for each of the basic  operations (or neurons) we want in our model. We only need to do this once per operation, thereby avoiding the need for manual differentiation everytime we change the architecture of our model.

The **forward** function calculates the output ($z$).

The **backward** function calculates the local gradients (e.g., $\frac{\partial z}{\partial x}$ and $\frac{\partial z}{\partial y}$).

All the rest is taken care of by applying the chain rule, as explained above.

In the example from before, we defined those two functions for a *Legendre polynomial of degree three*. Having defined this neuron/operation once, we can use it as a building block to build models that can be automatically differentiated.

PyTorch, of cause, comes with such an implementation for all the common building blocks, such as multiplication, addition, sigmoid, convolution, etc.